In [ ]:
############################################################################################
# IMPORTS
############################################################################################
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay


import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
#import tensorflow.keras as keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers


from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical



from IPython import display
from PIL import Image


import pathlib
import shutil
import tempfile
import concurrent

import pickle
import visualkeras

import json
import glob





print(tf.__version__)
print(tf.config.list_physical_devices())
############################################################################################

In [2]:
############################################################################################
# 1. CONSTANTS - PATHS
############################################################################################

#DATA_FS251 = './data/iFood_2019'
DATA_FS251 = './data/food-101'
CLASSES_FILE_NAME = '/formated_annot/classes_formated.csv'

TRAIN_INFO = '/annot/train_info.csv'
VAL_INFO = '/annot/val_info.csv'
TEST_INFO = '/annot/test_info.csv'

TRAIN_PICS_PATH = './data/iFood_2019/train_set/'
TEST_PICS_PATH = './data/iFood_2019/test_set/'
VAL_PICS_PATH = './data/iFood_2019/val_set/'

MODELS = './models/'

SEED = 111

#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
gpu_dev = tf.config.experimental.list_physical_devices('GPU')
for itm in gpu_dev:
    tf.config.experimental.set_memory_growth(itm, True)


############################################################################################
# IMPORTING DATA
############################################################################################
df_classes = pd.read_csv(DATA_FS251 + CLASSES_FILE_NAME)
df_train = pd.read_csv(DATA_FS251 + TRAIN_INFO, names=['file_name', 'class_num'])
df_validate = pd.read_csv(DATA_FS251 + VAL_INFO, names=['file_name', 'class_num'])
df_test = pd.read_csv(DATA_FS251 + TEST_INFO, names=['file_name'])

df_train.head(3)

In [3]:
############################################################################################
# 2. CONSTANTS - MODEL
############################################################################################
training_history = dict()

#N_TRAIN = len(df_train.iloc[:, 0])
EPOCHS = 30
BATCH_SIZE = 32              # 8 fit into GPU RAM, 64 fit into system RAM
RESIZE_TO = (300, 300)
#STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE

############################################################################################
# FUNCTIONS, DEFFINITIONS
############################################################################################

class CRelu(tf.keras.Layer):
    def __init__(self, axis=-1, **kwargs):
        self.axis = axis 
        super(CRelu, self).__init__(**kwargs)

    def build(self, input_shape):
        super(CRelu, self).build(input_shape)

    def call(self, x):
        x = tf.nn.crelu(x, axis=self.axis)
        return x

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        output_shape[-1] = output_shape[-1] * 2
        output_shape = tuple(output_shape)
        return output_shape

    def get_config(self, input_shape):
        config = {'axis': self.axis, }
        base_config = super(CReLU, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
############################################################################################
# import image files
############################################################################################

train_pics, val_pics = tf.keras.preprocessing.image_dataset_from_directory(
    './data/food-101/images/',
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=RESIZE_TO,
    shuffle=True,
    seed=SEED,
    validation_split=0.2,
    subset='both',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format='channels_last',
    verbose=True
)
train_pics = train_pics.batch(BATCH_SIZE).map(lambda x, y: (tf.keras.data_augmentation(x), y))

test_pics = tf.keras.preprocessing.image_dataset_from_directory(
    './data/food-101/images/',
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=RESIZE_TO,
    shuffle=True,
    seed=SEED,
    validation_split=0.15,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format='channels_last',
    verbose=True
)
#true_test_pics = tf.keras.preprocessing.image_dataset_from_directory(
#    TEST_PICS_PATH,
#    labels=None,
#    label_mode=None,
#    class_names=None,
#    color_mode='rgb',
#    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
#    shuffle=True,
#    seed=SEED,
#    validation_split=None,
#    subset=None,
#    interpolation='bilinear',
#    follow_links=False,
#    crop_to_aspect_ratio=False,
#    pad_to_aspect_ratio=False,
#    data_format='channels_last',
#    verbose=True
#)

In [ ]:
train_pics.class_names
#val_pics.as_numpy_iterator().next()[0]
#val_pics.take(1)

In [ ]:
############################################################################################
# DEFINING THE MODEL
############################################################################################
tf.keras.backend.clear_session()

core_model = tf.keras.applications.EfficientNetB3(include_top=False, input_shape=(300, 300, 3))
core_model.trainable = False
for layer in core_model.layers[-25:]:
    print(layer.output)
    print(layer.output)
#    layer.trainable = False

inputed = tf.keras.Input(shape=(300,300,3))
x = tf.keras.layers.Resizing(height = 300,width = 300,interpolation='bilinear',crop_to_aspect_ratio=False,pad_to_aspect_ratio=False,fill_mode='constant',fill_value=0.0)(inputed)
x = tf.keras.layers.RandomRotation(factor=0.2, seed=SEED)(x)
x = tf.keras.layers.RandomBrightness(factor=[-0.2, 0.2], seed=SEED)(x)
x = tf.keras.layers.RandomContrast(factor=0.2, seed=SEED)(x)
x = tf.keras.layers.RandomFlip(mode="horizontal", seed=SEED)(x)
x = core_model(x)

x = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
x = tf.keras.layers.Dropout(0.3)(x) 
outputed = tf.keras.layers.Dense(units=101,activation='softmax',activity_regularizer=tf.keras.regularizers.L2(), name='Output_layer')(x)


model_supclass = tf.keras.Model(inputed,outputed)

model_supclass.summary()


In [ ]:
############################################################################################
# COMPILING, FITTING 1
############################################################################################
model_supclass.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), #6.1 model, 6.5, 6.6
    #optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), #model 6.2, 6.3
    #optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), #model 6.4
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model_history_1 = model_supclass.fit(
    train_pics,
    validation_data=val_pics,
    epochs=20,
    callbacks=[tf.keras.callbacks.EarlyStopping(min_delta = 0, patience=5, restore_best_weights=True)]
)
############################################################################################
# COMPILING, FITTING 2
############################################################################################
core_model.trainable = True
for layer in core_model.layers[:-20]:
    layer.trainable = False
model_supclass.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model_history_2 = model_supclass.fit(
    train_pics,
    validation_data=val_pics,
    epochs=35,
    initial_epoch=5,
    callbacks=[tf.keras.callbacks.EarlyStopping(min_delta = 0, patience=5, restore_best_weights=True)]
)



In [8]:
############################################################################################
# SAVING MODEL
############################################################################################
MODEL_NAME = 'Nadine_food-101-EfNetB3-A0.__-earlystop-E__of45-B32_softCat_AugmPreprocLays_SEEDed(woDropout)_v1.2a'
os.makedirs( os.path.dirname(MODELS + 'test/' + MODEL_NAME + '/'), exist_ok=True)
model_supclass.save(MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + ".keras")
#model_supclass_loaded = load_model(MODELS + MODEL_NAME + '/'+ MODEL_NAME + ".keras")
#model_supclass_loaded.summary()

model_history_df = pd.concat( [ pd.DataFrame(model_history_1.history, columns=['accuracy','loss','val_accuracy','val_loss']), pd.DataFrame(model_history_2.history, columns=['accuracy','loss','val_accuracy','val_loss'])], axis=0).reset_index(drop=True)

# with pickle
with open(MODELS + 'test/'+ MODEL_NAME + '/'+ MODEL_NAME + ".pckl", 'wb') as file_pi:
    pickle.dump(model_history_df, file_pi)
# history in csv format
hits_df = []
hits_df = pd.DataFrame(model_history_df)
hits_df.to_csv(MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + '.hist_csv')


In [ ]:
############################################################################################
# EVALUATION - MODEL STRUCTURE IMAGES
############################################################################################
tf.keras.utils.plot_model(
    model_supclass,
    to_file=MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + '_model_diagram.png',
    show_shapes=True,
    show_dtype=True,
    show_layer_names=False,
    rankdir='TB',
    expand_nested=False,
    dpi=200,
    show_layer_activations=True,
    show_trainable=True
)


visualkeras.layered_view(model_supclass,
                        to_file=MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + '_layer_plot.png',
                        legend=True,
                        max_xy=250,
                        min_z = 1, 
                        max_z= 75,
                        scale_z = 5,
#                        color_map = color_map,
#                        show_nested = True,
                        spacing=50
#                        show_dimension=True
                        )
visualkeras.layered_view(core_model,
                        to_file=MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + '_core_layer_plot.png',
                        legend=True,
                        max_xy=250,
                        min_z = 1, 
                        max_z= 10,
                        scale_z = 5,
#                        color_map = color_map,
#                        show_nested = True,
                        spacing=5
#                        show_dimension=True
                        )

In [ ]:
############################################################################################
# EVALUATION - ACCURACY
############################################################################################

plt.legend(bbox_to_anchor = [1, 1.02])
plt.plot(model_history_1.history['accuracy'],label='Train accuracy1')
plt.plot(model_history_2.history['accuracy'],label='Train accuracy2')
plt.plot(model_history_1.history['val_accuracy'],label='Validation accuracy1')
plt.plot(model_history_2.history['val_accuracy'],label='Validation accuracy2')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + '_ACCURACY.png')
plt.show()

In [ ]:
############################################################################################
# EVALUATION - LOSS
############################################################################################

plt.legend(bbox_to_anchor = [1, 1.02])
plt.plot(model_history_1.history['loss'],label='Train loss 1')
plt.plot(model_history_2.history['loss'],label='Train loss 2')
plt.plot(model_history_1.history['val_loss'],label='Validation loss 1')
plt.plot(model_history_2.history['val_loss'],label='Validation loss 2')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig(MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + '_LOSS.png')
plt.legend()
plt.show()



In [ ]:
############################################################################################
# PREDICTION
############################################################################################

# Print out the score
score = model_supclass.evaluate(test_pics , batch_size=1, verbose=1)
print(score, model_supclass.metrics_names)

y_test_pred = np.array([])
y_test =  np.array([])
#y_test_pred_raw = np.empty(shape=(128,1))
#y_test_raw =  np.empty(shape=(128,1))
for x, y in test_pics:
        y_test_pred = np.concatenate([y_test_pred, np.argmax(model_supclass.predict(x, verbose=1), axis = -1)])
        #y_test_pred_raw = np.concatenate([y_test_pred, model_supclass.predict(x, verbose=0)], axis=0)
        y_test = np.concatenate([y_test, np.argmax(y.numpy(), axis=-1)])
        #y_test_raw = np.concatenate([y_test, y.numpy()], axis=0)

In [ ]:
############################################################################################
# PREDICTION - CONFMATRIX
############################################################################################

from sklearn.metrics import ConfusionMatrixDisplay
plt.rcParams['figure.figsize'] = (35, 35)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true= y_test, y_pred=y_test_pred)
#disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=np.unique(y_test))
labels = [str(val).lower().strip().replace('_',' ') for val in pd.DataFrame(val_pics.class_names, columns=['class'])['class'].tolist()]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot()
plt.xticks(rotation=90)
plt.savefig(MODELS + 'test/'  + MODEL_NAME + '/'+ MODEL_NAME + '_CONFMAT_l.png')
plt.show()






In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in val_pics.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(str( labels[i].numpy().astype(int) ) + str(np.argmax(model_supclass.predict(images[i], verbose=0), axis = -1))  )
        plt.axis("off")
        

In [ ]:
#plt.imshow(images[0].numpy().astype("uint8"))
#plt.show()
